In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import pandas as pd
import time

In [ ]:
path = '/kaggle/input/birdclef-2022/train_audio/'
bird_names = sorted(os.listdir(path))
print(bird_names)

In [ ]:
duration = 5
label = ['None']
number = 0
label_numbers = [number]
all_labels = np.zeros((1, len(bird_names)))
Magnitudes = np.array([np.zeros(110250)]) ## duration 5 seconds

In [ ]:
for bird_name in bird_names[0:3]:  ### RUNING ONLY IN THE TREE FIRST DUE TO TIME CONSUMING
    number +=1
    bird_tracks = os.listdir(path + bird_name)
    for track in bird_tracks:
        track_name = str(path + bird_name + '/' + track)
        y, sr = librosa.load(track_name, duration=duration, offset=0)
        if len(y) == 110250:
            print(bird_name)
            my_label_numbers = np.zeros((1, len(bird_names)))
            my_label_numbers[0, number-1] = 1
            all_labels = np.vstack((all_labels, my_label_numbers))
            label_numbers.append(number)
            label.append(bird_name)
            # sd.play(y, sr)
            dt = duration / len(y)
            time = np.arange(0, duration, dt)
            y_hat = np.fft.fft(y, len(y))
            PSD = y_hat * np.conj(y_hat)  ## power spectrum density
            Magnitudes = np.vstack((Magnitudes, PSD.real))
            n = len(time)
            freq = (1 / (dt * n)) * np.arange(n)
            frequency_name = freq
            L = np.arange(1, int(n / 2))  ##
            print(Magnitudes.shape, np.size(label))

In [ ]:
# df_label = pd.DataFrame(all_labels[1:], columns=bird_names)
# df_label.to_csv('birds.csv', index=False)

X = Magnitudes[1:, :]
print(X.shape)
#yl = label_numbers[1:-1]
yl = all_labels[1:, :]
print(yl.shape)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, yl, random_state=0)

model = RandomForestRegressor(max_leaf_nodes=150 ,random_state=0)
#model = DecisionTreeRegressor(max_depth=50)
#Fit Model
print("-------------------------------")
model.fit(train_X, train_y)
y_pred = model.predict(val_X)
print(y_pred)
print(model.score(val_X, val_y))

In [ ]:
test_track_path = '/kaggle/input/birdclef-2022/test_soundscapes/'
print(os.listdir(test_track_path))

In [ ]:
test_track = test_track_path + os.listdir(test_track_path)[0]
print(test_track)

In [ ]:
y, sr = librosa.load(test_track, offset=0)
duration = librosa.get_duration(y=y, sr=sr)
print(duration)
Test_Magnitudes = np.array([np.zeros(110250)])
end_time = [0]

In [ ]:
for offset in range(0, int(duration), 5):
    print(offset)
    end_time.append(offset+5)
    y, sr = librosa.load(test_track, duration=5, offset=offset)
#     print(y)
    y_hat = np.fft.fft(y, len(y))
    PSD = y_hat * np.conj(y_hat)  ## power spectrum density

    Test_Magnitudes = np.vstack((Test_Magnitudes, PSD.real))

In [ ]:
y_pred = model.predict(Test_Magnitudes[1:, :])

df = pd.DataFrame(y_pred, columns=bird_names)
df['ending_time'] = end_time[1:]

print(df)
df.to_csv('submission.csv', index=False)